## Use this Notebook to Hear Piano Performances Created by your Model

Use this notebook to generate, listen to, and save midi files of your models performances. Simply run the cells below in order.

In [1]:
from pianonet.core.note_array import NoteArray
from pianonet.training_utils.master_note_array import MasterNoteArray
from pianonet.training_utils.note_sample_generator import NoteSampleGenerator
from pianonet.model_building.get_model_input_shape import get_model_input_shape
import pianonet.model_inspection.performance_tools as pt

import os
import numpy as np
import random

from tensorflow.keras.models import load_model

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


### Load the Seed Dataset

In [2]:
master_note_array = MasterNoteArray(file_path='./pianonet_mini_dataset_0_validation.mna')
note_array_transformer = master_note_array.note_array_transformer

### Load the Trained Model

Make sure model path is pointing to the most recent model. This could be 1_trained_model, for example, if you restarted training.

In [3]:
run_path = "./"
model_path = "models/0_trained_model"

model = load_model(os.path.join(run_path, model_path))

### Generate a performance

Set `use_seed = False` if you want to hear a performance where the model starts with silence. This removes any human influence. If `use_seed = True`, the model starts its state as seeing a history of notes from a random segment in the validation set.

Increase `num_time_steps_in_performance` for longer performances.

In [4]:
use_seed = True
num_time_steps_in_performance = 600

num_time_steps_in_model_input = get_model_input_shape(model)/note_array_transformer.num_keys
num_time_steps_in_seed = int(num_time_steps_in_model_input) + 24


seed_note_array = master_note_array.get_note_array_from_random_segment_of_time_steps(num_time_steps=num_time_steps_in_seed)

if not use_seed:
    seed_note_array.array = np.zeros((len(seed_note_array.array),)).astype('bool')    

performance_note_array = pt.get_performance(model=model,
                                            seed_note_array=seed_note_array,
                                            num_time_steps=num_time_steps_in_performance,
                                            validation_fraction=0.0001)

Initializing state queues.
Resetting the state queues to the initial state (for a new performance).

==> Time step 0 seconds of audio is 0
==> Time step 48 seconds of audio is 1
==> Time step 96 seconds of audio is 2
==> Time step 144 seconds of audio is 3
==> Time step 192 seconds of audio is 4
==> Time step 240 seconds of audio is 5
==> Time step 288 seconds of audio is 6

Time per second of audio: 2.15 seconds
Timesteps added: 300.0


### Play the Performance

Context seconds is how much history is played back if a seed was used. Set to zero to hear where your model comes in. If you don't like how it sounds, add data or train longer! With enough data and training time, your model can sound like a pro.

In [7]:
p = performance_note_array.get_pianoroll()

context_seconds = 3

if not use_seed:
    p.trim_silence_off_ends()
else:
    num_input_time_steps = int(get_model_input_shape(model)/note_array_transformer.num_keys)    
    p = p[max(0, num_input_time_steps-48*context_seconds):]
    
p.play()

### Add Additional Timesteps to the Existing Performance

The cell below will add more model predictions to the existing performance. After it completes, run the cell above to play the extended performance.

In [6]:
num_time_steps_to_add = 600

performance_note_array = pt.get_performance(model=model,
                                            seed_note_array=performance_note_array,
                                            num_time_steps=num_time_steps_to_add,
                                            validation_fraction=0.0)

Initializing state queues.
Resetting the state queues to the initial state (for a new performance).

==> Time step 0 seconds of audio is 0
==> Time step 48 seconds of audio is 1
==> Time step 96 seconds of audio is 2
==> Time step 144 seconds of audio is 3
==> Time step 192 seconds of audio is 4
==> Time step 240 seconds of audio is 5
==> Time step 288 seconds of audio is 6
==> Time step 336 seconds of audio is 7
==> Time step 384 seconds of audio is 8
==> Time step 432 seconds of audio is 9
==> Time step 480 seconds of audio is 10
==> Time step 528 seconds of audio is 11
==> Time step 576 seconds of audio is 12

Time per second of audio: 2.513 seconds
Timesteps added: 600.0


### Save the Performance as a Midi File

If you like how it sounds, save it! That way you can show your friends. Midi files can be converted to mp3s using Garageband or online converters

In [8]:
file_name = "./my_models_awesome_performance.midi"
performance_note_array.get_pianoroll().save_to_midi_file(file_path=os.path.join(file_name))